# Automation System for Sending Email with Custom Certificates

For more info about sending email in Python, [**click here!**](https://realpython.com/python-send-email/) <br>
And for inserting text in images, [**click here**](https://amehta.github.io/posts/2019/08/adding-text-to-an-image-using-opencv-and-python/)

**Note** For only one receiver

## Import Modules

In [1]:
# Import module for image editing
from PIL import Image, ImageDraw, ImageFont
# Import modules for sending email
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
# Import markdown
import markdown

## Open Images

In [2]:
# Create image object
image = Image.open('images/Certificate for Gryffindor.jpg')
draw = ImageDraw.Draw(image)
# Image size
width,height = image.size

## Insert Participant Names

In [3]:
# Specified font-style and font-size 
font = ImageFont.truetype(font='font/OpenSans-CondBold.ttf',size=120)

In [4]:
# Specify text
text = 'Harry Potter'
# Text size for making it center
w,h = draw.textsize(text,font=font)

In [5]:
draw.text(xy=((width-w)/2,(height-h)/2 - 50),text=text,font=font,fill='black',align='left')

In [6]:
image.show()

## Convert PNG to PDF Document

In [7]:
imagePDF = image.convert('RGB')
imagePDF.save('Certificate for Gryffindor.pdf')

## Sending Email

In [21]:
sender_email = 'school@hogwarts.com'
receiver_email = 'harry_potter@hogwarts.com'
password = input('Type your password and press enter:')

Type your password and press enter:xxxxxxxxxxxxxxxx


**Configure subject of email**

In [9]:
subject = 'Certificate for Gryffindor Student - {receiver}'.format(receiver = text)

In [10]:
# Create a multipart message and set headers
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = receiver_email
message['Subject'] = subject
# message['Bcc'] = receiver_email # Recommended for mass emails 

**Configure body of email**

In [11]:
f = open('markdown/body.md','r')
htmlmarkdown = markdown.markdown(f.read())

In [12]:
print(htmlmarkdown)

<p>HOGWARTS SCHOOL of WITCHCRAFT and WIZARDRY  </p>
<p>Headmaster: Albus Dumbledore<br />
(Order of Merlin, First Class, Grand Sorc., Chf. Warlock,<br />
Supreme Mugwump, International Confed. of Wizards)  </p>
<p>Dear Mr/Ms. {receiver_name},<br />
We are pleased to inform you that you have been accepted at Hogwarts School of Witchcraft and Wizardry. Please find enclosed a list of all necessary books and equipment.<br />
Term begins on 1 September. We await your owl by no later than 31 July.</p>
<p>Yours sincerely,  </p>
<p>Minerva McGonagall<br />
Deputy Headmistress</p>


In [13]:
body = htmlmarkdown.format(receiver_name = text)

In [14]:
# Turn these into plain/html MIMEText objects
body = MIMEText(body,'html')

In [15]:
# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(body)

**Configure attachment file**

In [16]:
filename = 'Certificate for Gryffindor.pdf'
# Open PDF file in binary mode
with open(filename, 'rb') as attachment:
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
    part = MIMEBase('application','octet-stream')
    part.set_payload(attachment.read())

In [17]:
# Encode file in ASCII characters to send by email    
encoders.encode_base64(part)

In [18]:
# Add header as key/value pair to attachment part
part.add_header(
    'Content-Disposition',
    f'attachment; filename= {filename}'
)

In [19]:
# Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()

**Send email**

In [20]:
# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL('smtp.gmail.com',465,context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email,text
    )